In [ ]:
!pip install transformers torch gradio PyPDF2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00


In [3]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re

# Load model and tokenizer
model_name = "ibm-granite/granite-3.2-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Helper function for AI response
def generate_response(prompt, max_length=512, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    if torch.cuda.is_available():
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()
    return response

# Concept Explanation
def explain_concept(concept):
    if not concept.strip():
        return "Please enter a concept to explain."
    prompt = f"Explain the concept of {concept} in simple and clear terms."
    return generate_response(prompt, max_length=400)

# Quiz Generator with 4 options
def generate_quiz(concept):
    if not concept.strip():
        return "Please enter a concept to generate a quiz."
    prompt = f"""
    Create one multiple-choice quiz question about {concept}.
    Provide exactly 4 options (A, B, C, D) and mention the correct answer at the end.

    Format:
    Question: ...
    A) ...
    B) ...
    C) ...
    D) ...
    Correct Answer: ...
    """
    return generate_response(prompt, max_length=500)

# Gradio App
with gr.Blocks(title="Educational AI Assistant") as app:
    gr.Markdown("# 🎓 Educational AI Assistant")

    with gr.Tabs():
        with gr.TabItem("Concept Explanation"):
            concept_input = gr.Textbox(label="Enter a Concept", placeholder="e.g., Machine Learning")
            explain_btn = gr.Button("Explain")
            explanation_output = gr.Textbox(label="Explanation", lines=10)
            explain_btn.click(explain_concept, inputs=concept_input, outputs=explanation_output)

        with gr.TabItem("Quiz Generator"):
            quiz_input = gr.Textbox(label="Enter a Concept for Quiz", placeholder="e.g., Artificial Intelligence")
            quiz_btn = gr.Button("Generate Quiz")
            quiz_output = gr.Textbox(label="Quiz Question with 4 Options", lines=10)
            quiz_btn.click(generate_quiz, inputs=quiz_input, outputs=quiz_output)

app.launch(share=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f08b37066a6900ad86.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
